In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import scipy.linalg
import scipy.fftpack
import scipy.optimize
import math
import time

In [21]:
p_gen = np.polynomial.chebyshev.Chebyshev

In [25]:
def calc_power_sums(xs, k):
    return np.array([
        np.sum(xs**i) for i in range(k)
    ])
def calc_cheby_sums(xs, k):
    return np.array([
        np.sum(p_gen.basis(i)(xs)) for i in range(k)
    ])

In [17]:
maxk = 11
cheby_coeffs = [
    np.array([1]),
    np.array([0,1]),
]
for i in range(2,maxk+1):
    Tn1 = cheby_coeffs[-1]
    Tn2 = cheby_coeffs[-2]
    cheby_coeffs.append(
        np.insert(2*Tn1,0,0) 
        - np.concatenate([Tn2,[0,0]])
    )

In [19]:
def shift(
    ms, xmin, xmax
):
    k = len(ms) - 1
    r = (xmax - xmin) / 2
    xc = (xmax + xmin) / 2
    ms_scaled = np.zeros(k+1)
    nxc_powers = np.power(-xc, np.arange(0,k+1))
    for m in range(k+1):
        ms_scaled[m] = np.sum(
            scipy.special.binom(m, np.arange(0,m+1)) 
            * nxc_powers[:m+1][::-1]
            * ms[:m+1]
        ) * math.pow(r,-m)
    return ms_scaled
def shifted_to_cheby(
    ms_scaled
):
    k = len(ms_scaled) - 1
    ms_cheby = np.zeros(k+1)
    for i in range(k+1):
        ms_cheby[i] = np.inner(
            cheby_coeffs[i], ms_scaled[:i+1]
        )
    return ms_cheby

In [91]:
x1 = np.linspace(0,1000,1001)
s1 = calc_power_sums(x1, 4).astype(int)
print(repr(s1))
print(shifted_to_cheby(shift(s1, np.min(x1), np.max(x1)))/1001.0)

array([        1001,       500500,    333833500, 250500250000])
[ 1.     0.    -0.332  0.   ]


In [93]:
x2 = np.linspace(-1,1,1001)
calc_cheby_sums(x2, 4)/1001.0

array([  1.00000000e+00,   2.83933161e-17,  -3.32000000e-01,
         2.83933161e-17])

In [67]:
df = pd.read_csv("../sampledata/uci_retail_cleaned.csv")
data = np.array(df["x"],dtype="float64")

In [77]:
dmin = np.min(data)
dmax = np.max(data)
dc = (dmax+dmin)/2
dr = (dmax-dmin)/2
ndata = (data - dc) /dr
calc_cheby_sums(ndata, 10)[6]

509831.77099894051

In [78]:
s1 = calc_power_sums(data, 10)
shifted_to_cheby(shift(
    s1, np.min(data), np.max(data)))[6]

509831.77099894546